## My Trainer

In [1]:
%cd /home/q123/Desktop/explo/

from mushroom_rl.environments.dm_control_env import DMControl
from mushroom_rl.policy import DeterministicPolicy
from src.ddpg import DDPG
from src.helpers import setup_experiment
from src.config import get_configs
import torch
from src.approximators.actor import ActorNetwork
from src.approximators.critic import CriticNetwork

import torch.nn.functional as F

# MDP
horizon = 500
gamma = 0.99
gamma_eval = 1.
#mdp = DMControl('walker', 'stand', horizon, gamma)


# Policy 

policy_class = DeterministicPolicy
policy_params = dict()

# Settings
initial_replay_size = 500
max_replay_size = 5000
batch_size = 200
n_features = 80
tau = .001

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/envs/registration.py:415: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
pybullet build time: Jun 23 2022 12:25:14


In [2]:
env_name = "Swimmer-v4"
kernel_name = "rbf"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
_,env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers=[])


mdp = env.env
# Approximator
actor_input_shape = mdp.info.observation_space.shape
actor_params = dict(network=ActorNetwork,
                    n_features=n_features,
                    input_shape=actor_input_shape,
                    output_shape=mdp.info.action_space.shape)

actor_optimizer = {'class': torch.optim.Adam,
                   'params': {'lr': 1e-5}}

critic_input_shape = (actor_input_shape[0] + mdp.info.action_space.shape[0],)
critic_params = dict(network=CriticNetwork,
                     optimizer={'class': torch.optim.Adam,
                                'params': {'lr': 1e-3}},
                     loss=F.mse_loss,
                     n_features=n_features,
                     input_shape=critic_input_shape,
                     output_shape=(1,))

# Agent
agent = DDPG(mdp.info, policy_class, policy_params,
             actor_params, actor_optimizer, critic_params,
             batch_size, initial_replay_size, max_replay_size,
             tau)

MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]


/home/q123/Desktop/explo/src/environments/objective.py:204: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rslt = torch.tensor(state, dtype=torch.float32)
/home/q123/Desktop/explo/src/environments/objective.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states.append(torch.tensor(self.manipulate_state(state)))
/home/q123/Desktop/explo/src/environments/objective.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  actions.append(torch.tensor(action))


Using ard_num_dims = 18


In [3]:
import multiprocessing as mp
from copy import deepcopy
import torch


class ESQOptimizer(object):
    
    
    def __init__(self,states,critic,actor_mlp,actor_params,
                        sigma,params_per_step,n_workers=None):
        
        if n_workers is None : n_workers = mp.cpu_count()
        
        self.args = locals()
        self.args.pop("self")
        
        self.actor_params = actor_params
        self.optimizer = torch.optim.Adam([actor_params])
        
         
    def advantage_gradient(self,states,critic,actor_mlp,actor_params):
        
        noisy_actions = actor_mlp(actor_params,states).squeeze().T
        noisy_q = critic(states,noisy_actions,output_tensor=True) ##  add absorbing flag
        noisy_q = torch.sum(noisy_q)
        noisy_grad = torch.autograd.grad(noisy_q,actor_params)
        
        return noisy_grad

    def run_noisy_advantage(self,states,critic,actor_mlp,actor_params,
                            sigma,seed):
        
        torch.manual_seed(seed)
        eps = torch.randn_like(actor_params) * sigma
        
        actor_params1 = actor_params + eps
        actor_params2 = actor_params - eps
        
        #grad1 = self.advantage_gradient(states,critic,actor_mlp,actor_params1)
        #grad2 = self.advantage_gradient(states,critic,actor_mlp,actor_params2)
        noisy_actions = actor_mlp(actor_params1,states).squeeze().T
        noisy_q = critic(states,noisy_actions,output_tensor=True) ##  add absorbing flag
        noisy_q = torch.sum(noisy_q)
        noisy_grad = torch.autograd.grad(noisy_q,actor_params)[0] ## hotfix
        
        #return grad1+grad2
        
        return noisy_grad
        
    def run_parallel_advantage(self,states,critic,actor_mlp,actor_params,
                               sigma,params_per_step,n_workers):
        
        
    
        args = [(states,critic,actor_mlp,actor_params,sigma,seed) for seed in range(params_per_step)]
        
        ctx = mp.get_context('fork')
        
        # Step 1: Init multiprocessing.Pool()

        with ctx.Pool(n_workers) as pool:
        
            # Step 2:  Run processes (we might need to use mapreduce to avoid big memory usage)
            grads = pool.starmap(self.run_noisy_advantage,args) ## list of [(reward*eps)]

            # Step 3: Wait for workers to run then close pool
            pool.close()
            pool.join()

        return grads
    
    def compute_grads(self):
        
        grads = self.run_parallel_advantage(**self.args)
        
        grads = torch.sum(torch.stack(grads),dim=0)
        
        return grads

    
    def step(self):
        
        self.optimizer.zero_grad()  
        actor_grad = self.compute_grads()      
        self.actor_params.grad = -actor_grad ## optimizer usually minimizes
        self.optimizer.step()
        
        print(self.actor_params)     

In [4]:
%cd /home/q123/Desktop/explo/src
#from src.optimizers.esq_pytorch import ESQOptimizer

states = torch.rand(10,8)
parameters = torch.rand(1,18,requires_grad=True)
optimizer = ESQOptimizer(states=states,
                         critic=agent._critic_approximator,
                         actor_mlp =env.mlp,
                         actor_params = parameters,
                         sigma=1,
                         params_per_step=4,
                         n_workers=2)

for i in range(10):
    
    grads = optimizer.step()

/home/q123/Desktop/explo/src
tensor([[0.3654, 0.7330, 0.4882, 0.2555, 0.8375, 0.0195, 0.2320, 0.2841, 0.1937,
         0.6388, 0.5567, 0.4018, 0.2184, 0.4540, 0.5587, 0.3721, 0.8128, 0.8150]],
       requires_grad=True)
tensor([[0.3664, 0.7340, 0.4892, 0.2565, 0.8385, 0.0205, 0.2330, 0.2851, 0.1947,
         0.6398, 0.5577, 0.4028, 0.2194, 0.4550, 0.5597, 0.3731, 0.8138, 0.8160]],
       requires_grad=True)
tensor([[0.3674, 0.7350, 0.4902, 0.2575, 0.8395, 0.0215, 0.2341, 0.2861, 0.1957,
         0.6408, 0.5587, 0.4038, 0.2204, 0.4560, 0.5607, 0.3741, 0.8148, 0.8170]],
       requires_grad=True)
tensor([[0.3684, 0.7360, 0.4912, 0.2585, 0.8405, 0.0225, 0.2351, 0.2872, 0.1967,
         0.6418, 0.5597, 0.4048, 0.2214, 0.4570, 0.5617, 0.3751, 0.8158, 0.8180]],
       requires_grad=True)
tensor([[0.3694, 0.7371, 0.4922, 0.2595, 0.8415, 0.0235, 0.2361, 0.2882, 0.1977,
         0.6428, 0.5607, 0.4058, 0.2224, 0.4580, 0.5627, 0.3761, 0.8168, 0.8190]],
       requires_grad=True)
tensor([[0.3704,

In [5]:


def critic_step(transitions):

    agent._replay_memory.add(transitions)

    if agent._replay_memory.initialized:
        
            state, action, reward, next_state, absorbing, _ =\
                agent._replay_memory.get(agent._batch_size())

            q_next = agent._next_q(next_state, absorbing)
            q_target = reward + agent.mdp_info.gamma * q_next

            agent._critic_approximator.fit(state, action, q_target,
                                            **agent._critic_fit_params)
            
            agent._update_target(self._critic_approximator,
                                self._target_critic_approximator)

In [6]:
grads[0]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
states = torch.rand(10,8)
parameters = torch.rand(1,18,requires_grad=True)
actions = env.mlp(parameters,states).squeeze().T
q = agent._critic_approximator(states,actions,output_tensor=True)
adv = torch.sum(q)
torch.autograd.grad(adv,parameters)

(tensor([[0.8204, 0.8464, 0.9284, 1.4442, 0.9304, 0.9537, 0.9842, 0.9743, 1.5538,
          1.6853, 1.6955, 2.4404, 1.8310, 1.9602, 2.0126, 1.7671, 1.8853, 3.5630]]),)

In [ ]:
# from mushroom_rl.approximators import Regressor
# from mushroom_rl.approximators.parametric import TorchApproximator
# from mushroom_rl.utils.replay_memory import ReplayMemory

# replay_memory = ReplayMemory(initial_replay_size, max_replay_size)
# critic_approximator = Regressor(TorchApproximator,
#                                               **critic_params)


In [ ]:
# optimizer = ESOptimizer(env,torch.zeros(env.mlp.len_params),sigma=1e-2,
#                 params_per_step=50,episodes_per_param=1,n_workers=8)
# # for i in range(1):
    
# #     optimizer.step()
    
# #     if i % 3 == 0:
# #         avg_reward,_,transitions = env.run_many(optimizer.policy_params,5)
# #         # print(f'avg_rewarad {avg_reward} ')
# #         # print(f'policy_params : {optimizer.policy_params}')
